Style GAN 2를 pytorch로 구현!

In [ ]:
import math
from typing import Tuple, Optional, List

import numpy as np
import torch
import torch.nn.functional as F
import torch.utils.data
from torch import nn

# Mapping Network를 구성하기!

class MappingNetwork(nn.Module):
  """
  Mapping Network

  MLP with 8 linear layers
  mapping network는 latent vector를 중간 단계의 latent space로 보내고,
  그 space는 image space에서 특정 값들과 연계됨
  """

  def __init__(self, features: int, n_layers: int):
    """
    'features'는 특성의 갯수
    'n_layers'는 mapping network의 layer의 갯수
    """
    super().__init__()

    # MLP 생성
    layers = []
    for i in range(n_layers):
      layers.append(EqualizedLinear(features, features))
      layers.append(nn.LeakyReLU(negative_slope=0.2, inplace=True))

    self.net = nn.Sequential(*layers)

  def forward(self, z: torch.Tensor):
    # 표준화
    z = F.normalize(z, dim=1)
    #
    return self.net(z)

class EqualizedLinear(nn.Module):
  """
  Learning-rate Equalized Linear Layer

  """

  def __init__(self, in_features: int, out_features: int, bias: float = 0.):
    super().__init__()
    # Learning rate를 weight와 같게...
    self.weight = EqualizedWeight([out_features, in_features])
    # Bias
    self.bias = nn.Parameter(torch.ones(out_features)*bias)

  def forward(self, x: torch.Tensor):
    # linear transform
    return F.linear(x, self.weight(), bias=self.bias)


class EqualizedWeight(nn.Module):
  """
  optimizer는 learning rate에 따라 적용되나
  effective weights는 learning rate에 따라 적용되지 않음
  equalized learning rate없이는 효율적으로 학습이 되지 않음

  """

  def __init__(self, shape: List[int]):
    """
    'shape': weight parameter의 shape
    """
    super().__init__()

    # 상수 초기화
    self.c = 1/ math.sqrt(np.prod(shape[1:]))
    # weight 초기화
    self.weight = nn.Parameter(torch.randn(shape))

  def forward(self):
    # weight, constance 곱해서 돌려줌..
    return self.weight * self.c



In [ ]:

class StyleBlock(nn.Module):
  """
  Style Block
  noise를 더하고, 그림의 품질, 그리고 스타일을 지정하는 block
  weight modulation, convolution layer를 갖고 있음
  """

  def __init__(self, d_latent: int, in_features: int, out_features: int):
    """
    d_latent - dimensionality of weight
    in_features - input feature map의 feature 숫자
    out_features - output feature map의 feature 숫자
    """

    super().__init__()
    # style vector를 weight에서 얻기
    self.to_style = EqualizedLinear(d_latent, in_features, bias=1.0)
    # wegiht modulated convolution layer
    self.conv = Conv2dWeightModulate(in_features, out_features, kernel_size=3)
    # noise scale
    self.scale_noise = nn.Parameter(torch.zeros(1))
    # Bias
    self.bias = nn.Parameter(torch.zeros(out_features))
    # activation function
    self.activation = nn.LeakyReLU(negative_slope=0.2, inplace=True)


  def forward(self, x: torch.Tensor, w: torch.Tensor, noise: Optional[torch.Tensor]):
    """
    x 는 input feature map의 shape '[batch_size, in_features, height, width]'
    w 는 weight와 shape '[batch_size, d_latent]'
    noise는 tensor의 shape '[batch_size, 1, height, width]'
    """

    # Style Vector 얻기
    s = self.to_style(w)
    # weight modulated convolution
    x = self.conv(x, s)
    # scale + noise
    if noise is not None:
      x = x + self.scale_noise[None, :, None, None] * noise
    # bias더하고, activation function
    return self.activation(x + self.bias[None, :, None, None])


class Conv2WeightModulate(nn.Module):
  """
  Convolution with weight modulation and Demodulation
  해당 레이어는 convolution weights를 style vector와 scale을 하고, demodulates를 normalize로 나누어 진행
  """

  def __init__(self, in_features: int, out_features: int, kernel_size:int, demodulate: bool=True, eps: float = 1e-8):
    """
    in_feature는 input feature map의 feature 숫자
    out_feature는 output feature map의 feature 숫자
    kernel_size는 convolution kernel의 크기
    demodulate는 normalize weight를 표준편차에 맞게 재조정할지 말지
    eps는 normalizing에 활용되는 epsilon
    """
    super().__init__()
    #
    self.out_features = out_features

    self.demodulate = demodulate
    self.padding = (kernal_size-1) // 2
    # learning rate와 함께 조정되는 weight parameter
    self.weight = EqualizedWeight([out_features, in_features, kernel_size, kernel_size])

    self.eps = eps

  def forward(self, x: torch.Tensor, s: torch.Tensor):
    """
    x 는 input feature map의 shape '[batch_size, in_features, height, width]'
    s 는 style기반의 scaling tensor의 shape '[batch_size, in_features]'

    """
    # batch size, height, width
    b, _, h, w = x.shape

    # scale 형태 재조정
    s = s[:, None, :, None, None]
    # learning rate에 맞게 조정된 weights vector
    weights = self.weight()[None, :, :, :, :]

    # input channel, output channel, kernel index
    # 결과로 얻는 shape은 '[batch_size, out_features, in_features, kernel_size, kernel_size]'
    weights = weights * s

    # demodulate
    if self.demodulate:
      # $$\sigma_j = \sqrt{\sum_{i, k} {w1 _{i, j, k}}^2 + \epsilon}$$
      sigma_inv = torch.rsqrt((weights ** 2).sum(dim=(2,3,4), keepdim=True) + self.eps)

      weights = weights * sigma_inv

    # x reshape
    x = x.reshape(1, -1, h, w)

    # reshape weight
    _, _, *ws = wieghts.shape
    weights = weights.reshape(b * self.out_features, *ws)

    # convolution을 계산할 때 그룹지어 계산하면 계산이 효율적임
    # 그러나, batch에 있는 sample의 kenel weights가 다름
    x = F.conv2d(x, weights, padding=self.padding, groups=b)

    # x를 '[batch_size, out_features, height, width]'로 형태 재구성
    return x.reshape(-1, self.out_features, h, w)


In [ ]:
#
class GenerationBlock(nn.Module):
  """
  그림을 만드는 부분
  $A$는 linear layer
  $B$는 해상도를 올리고, 주변에 전파하는 역할을 담당 (noise는 단일 채널로 동작)
  ['toRGB'] 는 간단한 style modulation을 갖고 있음
  generator block은 2개의 style block과 하나의 RGB Block으로 구성됨
  style block은 style modulation을 가진 3개의 convolution layer로 구성됨
  """

  def __init__(self, d_latent: int, in_features: int, out_features: int):
    """
    'd_latent'는 weights의 차원
    'in_features'는 입력 특성맵의 갯수
    'out_features'는 출력 특성맵의 갯수
    """
    super().__init__()

    # 첫번째 style block은 feature map의 크기를 out_features의 크기로 만들어줌
    self.style_block1 = StyleBlock(d_latent, in_features, out_features)
    # 두 번째 style block
    self.style_block2 = StyleBlock(d_latent, out_features, out_features)

    # toRGB block
    self.to_rgb = ToRGB(d_latent, out_features)


  def forward(self, x:torch.Tensor, w:torch.Tensor, noise: Tuple[Optional[torch.Tensor], Optional[torch.Tensor]]):
    """
    x 는 input feature map의 shape '[batch_size, in_features, height, width]'
    w 는 weight와 shape '[batch_size, d_latent]'
    noise는 두 noise tensors의 shape '[batch_size, 1, height, width]'
    """

    # 첫번째 style block은 첫 번째 noise tensor와 함께 동작
    # 출력은 [batch_size, out_features, out_features, height, width]의 형태
    x = self.style_block1(x, w, noise[0])
    # 두 번째 style block은 두 번째 noise tensor와 함께 동작
    # 출력은 [batch_size, out_features, out_features, height, width]의 형태
    x = self.style_block2(x, w, noise[1])

    # toRGB block
    rgb = self.to_rgb(x, w)

    # 특성 맵과 rgb 그림을 같이 돌려줌
    return x, rgb

In [ ]:
# toRGB
class ToRGB(nn.Module):
  """
  feature map을 이용하여 RGB 그림을 만듦, 1개의 convolution layer를 가짐
  """
  def __init__(self, d_latent:int, features: int):
    """
    d_latent: weights의 차원
    features: feature map의 갯수
    """

    super().__init__()
    # weight modulated convolution layer
    self.to_style = EqualizedLinear(d_latent, features, bias=1.0)

    # weight modulated convolution layer - demolation이 없는 layer
    self.conv = Conv2dWeightModulate(features, 3, kernel_size=1, demodulate=False)

    # bias
    self.bias = nn.Parameter(torch.zeros(3))

    # activation function
    self.activation = nn.LeakyReLU(negative_slope=0.2, inplace=True)

  def forward(self, x:torch.Tensor, w:torch.Tensor):
    """
    x 는 input feature map의 shape '[batch_size, in_features, height, width]'
    w 는 weight와 shape '[batch_size, d_latent]'
    """

    # style vector 얻기
    style = self.to_style(w)
    # weight modulated convolution
    x = self.conv(x, style)
    # bias와 evaluate activation function 적용
    return self.activation(x + self.bias[None, :, None, None])



In [ ]:
# generator
class Generator(nn.Module):
  """
  Style GAN 2의 Generator

  A는 linear layer로
  B는 주변에 전파 및 규모를 키우는 역할을 수행(noise는 단일 채널)
  toRGB는 style modulation을 갖고 있음

  generator는 이미 학습한 상수로부터 시작
  block들로 구성되어있는데, feature map의 해상도는 각 블록을 통과할 때마다 두 배로 증가
  각 block은 rgb image를 출력하고, 크기가 증가되어, 최종 rgb 그림을 생성

  """

  def __init__(self, log_resolution: int, d_latent: int, n_features: int=32, max_features: int=512):

    # '[512, 512, 256, 128, 64, 32]
    features = [min(max_features, n_features*(2** i)) for i in range(log_resolution-2, -1, -1)]
    # generator block의 숫자
    self.n_blocks = len(features)

    # 훈련 가능한 4종류의 상수
    self.initial_constant = nn.Parameter(torch.randn((1, features[0], 4, 4)))

    # 첫 번째 style block - 4배로 키워줌
    self.style_block = StyleBlock(d_latent, features[0], features[0])
    self.to_rgb = ToRGB(d_latent, features[0])
    # Generator block
    blocks = [GeneratorBlock(d_latent, features[i-1], features[i]) for i in range(1, self.n_blocks)]
    self.blocks = nn.ModuleList(blocks)

    # 2배씩 증가하는 layer들 feature들은 각각 block 단위로 추출
    self.up_sample = UpSample()


  def forward(self, w: torch.Tensor, input_noise: List[Tuple[Optional[torch.Tensor], Optional[torch.Tensor]]]):
    """
    w 는 weight와 shape '[n_blocks, batch_size, d_latent]'
    서로 다른 layer에서 사용하는 서로 다른 style을 섞기 위해 각 generator block마다 서로 구분된 w를 가짐

    input_noise는 각 block마다 별개의 noise
    noise sensors의 쌍으로 된 리스트는 각 block(최초의 block을 제외하고)마다 두 noise inputs를 갖고, convolution layer로 전달하기 때문

    """

    # batch size 얻기
    batch_size = w.shape[1]

    # learned constant를 batch size에 맞게 확장
    x = self.initial_constant.expand(batch_size, -1, -1, -1)

    # 첫번째 style block
    x = self.style_block(x, w[0], input_noise[0][1])

    # 첫번째 rgb 그림
    rgb = self.to_rgb(x, w[0])

    # 나머지 block 평가
    for i in range(1, self.n_blocks):
      # 특성 맵을 추출하여 크기 키우기
      x = self.up_sample(x)
      # generator block을 통과
      x, rgb_new = self.blocks[i-1](x, w[i], input_noise[i])
      # rgb image를 추출하고, 크기 키우기, 그리고 block에서 얻은 rgb 더하기
      rgb = self.up_sample(rgb) + rgb_new

    # 최종 그림 결과 돌려주기
    return rgb

In [ ]:
# Discriminator Block

class DiscriminatorBlock(nn.Module):
  """
  discriminator block

  2개의 $3 \times 3$ convolution layer가 residual connection을 갖고 있음

  """

  def __init__(self, in_features, out_features):
    """
    in_features - input feature map의 feature 숫자
    out_features - output feature map의 feature 숫자
    """
    super().__init__()
    # 2개의 convolution

    self.residual = nn.Sequential(DownSample(), EqualizedConv2d(in_features, out_features, kernel_size=1))

    self.block = nn.Sequential(
        EqualizedConv2d(in_features, out_features, kernel_size=3, padding=1),
        nn.LeakyReLU(negative_slope=0.2, inplace=True),
        EqualizedConv2d(out_features, out_features, kernel_size=3, padding=1),
        nn.LeakyReLU(negative_slope=0.2, inplace=True)
    )

    self.down_sample = DownSample()

    # 확장 파라미터
    self.scale = 1/ math.sqrt(2)

  def forward(self, x):
    residual = self.residual(x)
    # convolution
    x = self.block(x)
    # Downsample
    x = self.down_sample(x)
    return (x + residual) * self.scale


In [ ]:
# discriminator
class Discriminator(nn.Module):
  """
  Discriminator - image > feature map

  """
  def __init__(self, log_resolution: int, n_features: int = 64, max_features: int = 512):
    """
    'log_resolution' - log_2 image resolution
    'n_features' - first block을 통해 뽑아내는 가장 높은 해상도의 feature
    'max_features' - generator block에 있는 가장 높은 features
    """

    super().__init__()
    # RGB 그림 > features map
    self.from_rgb = nn.Sequential(
        EqualizedConv2d(3, n_features, 1),
        nn.LeakyReLU(negative_slope=0.2, inplace=True)
    )
    # 각 block에 들어갈 features 계산
    features = [min(max_features, n_features * (2**i)) for i in range(log_resolution-1)]
    # Discriminator의 block 갯수
    n_blocks = len(featuers) - 1
    # Discriminator block
    blocks = [DiscriminatorBlock(features[i], features[i+1]) for i in range(n_blocks)]

    self.blocks = nn.Sequential(*blocks)
    # Mini-batch 표준편차
    self.std_dev = MinibatchStdDev()
    # 표준편차를 도입하고 난 뒤의 features 숫자
    final_features = features[-1] +1
    # Final convolution layer
    self.conv = EqualizedConv2d(2*2*final_features, 1)
    # Final linear layer - 분류를 위해 도입되는 최종
    self.final = EqualizedLinear(2*2*final_features, 1)

  def forward(self, x: torch.Tensor):
    """
    'x' - 입력된 그림의 모습 [batch_size, 3, height, width]
    """

    x = x - 0.5
    # rgb그림을 입력 데이터 형태로 변경
    x = self.from_rgb(x)
    # discriminator block
    x = self.blocks(x)
    # mini-batch의 분산 계산, mini-batch표준편차 계산하고 붙이기
    x = self.std_dev(x)
    # convolution layer 통과
    x = self.conv(x)
    # 펼치기...
    x = x.reshape(x.shape[0], -1)
    # 분류 확률 계산
    return self.final(x)



In [ ]:
# path length regularization
# 조금 더 random성을 부여하여... 그림에 랜덤성 등을 추가할 것

class PathLengthPenalty(nn.Module):
  """
  Path Length Penalty

  이 재정규화 방식은 그림의 크기, 변화되는 값을 고정화 시켜 줌 > 그러니까.. 그림을 그려주거나 변주를 줄 때 지나친 연산을 막고, 그 변화의 방향성의 스텝 같은 것등을

  $$\mathbb{E}_{w \sim f(z), y \sim \mathcal{N}(0, \mathbf{I})}
    \Big(\Vert \mathbf{J}^\top_{w} y \Vert_2 - a \Big)^2$$

  $\mathbf{J}_w$ 는 Jacobian
  $\mathbf{J}_w = \frac{\partial g}{\partial w}$,
  $w$는 \w \in \mathcal{W}$에서 mapping network로 뽑은 것
  $y$는 noise가 섞인 그림 $\mathcal{N}(0, \mathbf{I})$

  $a$는 지수적으로 움직이는 평균 $\Vert \mathbf{J}^\top_{w} y \Vert_2$ 훈련 과정 중에 일어나는 움직임을 제어하는 파라미터 같음

  $\mathbf{J}^\top_{w} y$는 Jacobian의 계산을 제외하고 계산한 것
  $$\mathbf{J}^\top_{w} y = \nabla_w \big(g(w) \cdot y \big)$$
  """

  def __init__(self, beat: float):



  def forward(self, w: torch.Tensor, x: torch.Tensor):
    """
    'w'는 batch의 모습 '[batch_size, d_latent]'
    'x'는 생성되는 그림의 모습 '[batch_size, 3, height, width]'
    """
    # 돌릴 device
    device = x.device

    # pixel갯수
    image_size = x.shape[2] * x.shape[3]
    # $y 계산 \in \mathcal{N}(0, \mathbf{I})$
    y = torch.randn(x.shape, device=device)

    # 그림 크기 계산하기
    # 해당 방식은 논문에서 언급되었으나,
    output = (x*y).sum() / math.sqrt(image_size)

    # gradient 계산
    gradients, *_ = torch.autograd.grad(outputs=output,
                                        inputs=w,
                                        grad_outputs=torch.ones(output.shape, device=device),
                                        create_graph=True)
    # L2-norm 계산
    norm = (gradient **2).sum(dim=2).mean(dim=1).sqrt()

    # 첫번째 step 정규화
    if self.steps>0:
      # a 값 계산
      a = self.exp_sum_a / (1-self.beta ** self.steps)
      # 페널티 계산
      loss = torch.mean((norm - a) ** 2)
    else:
      loss = norm.new_tensor(0)
    # 평균 계산
    mean = norm.mean().detach()
    # exponental sum 더하기
    self.exp_sum_a.mul_(self.beta).add_(mean, alpha=1 - self.beta)
    #스텝 진행 상태 더하기
    self.steps.add_(1.)
    #페널티
    return loss






In [ ]:
# 모델 훈련, 평가
import math
from pathlib import Path
from typing import Integer, Tuple
import numpy as np
import os
import torch
import torch.utils.data
import torchvision
from PIL import Image

from torchvision import datasets, transforms, utils

from label_nn.gan.stylegan import Discriminator, Generator, MappingNetwork, GradientPenalty, PathLengthPenalty
from label_nn.gan.wasserstein import DiscriminatorLoss, GeneratorLoss
from label_nn.utils import cycle_dataloader

from tqdm.notebook import tqdm

In [1]:
class Dataset(torch.utils.data.Dataset):
  """
  data set
  """

  def __init__(self, path: str, image_size: int):
    """
    'path'는 경로 지정
    'image_size'는 그림 크기
    """
    super().__init__()

    # '.jpg' 파일의 경로 확인
    self.paths = [p for p in Path(path).glob(f'**/*.jpg')]

    self.transform = torchvision.transforms.Compose([
        # 그림 크기 변경
        torchvision.transforms.Resize((image_size, image_size)),
        # tensor로 변경
        torchvision.transforms.ToTensor(),
    ])

  def __len__(self):
    """그림의 갯수"""
    return len(self.paths)

  def __getitem__(self, index):
    """'index'번째의 그림"""
    path = self.paths[index]
    img = Image.open(path)
    return self.transform(img)


SyntaxError: incomplete input (<ipython-input-1-9527a0d3ce6e>, line 7)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [ ]:
dataset_path: str = os.path.join('data_faces', 'img_align_celeba')

# batch_size
batch_size: int = 32
# Dimension
d_latent: int = 512

image_size: int = 64
# mapping network에 쓰일 layer의 갯수
mapping_network_layers: int = 8

In [ ]:
# [Gradient Penalty Regularization Loss](index.html#gradient_penalty)
gradient_penalty = GradientPenalty()
# Gradient penalty coefficient
gradient_penalty_coefficient: float = 10.0

# [Path length penalty]()
path_length_penalty: PathLengthPenalty

In [ ]:
# 초기화

dataset = Dataset(dataset_path, image_size)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)

loader = cycle_dataloader(dataloader)

In [ ]:
log_resolution = int(math.log2(image_size))

# discriminator, generator 객체 만들기
discriminator = Discriminator(log_resolution).to(device)
generator = Generator(log_resolution, d_latent).to(device)
# Generator block에서 생성하는 꼴과 잡티 입력
n_gen_blocks = generator.n_blocks
# mapping_network 생성
mapping_network = MappingNetwork(d_latent, mapping_network_layers).to(device)
# path length penalty
path_length_penalty = PathLengthPenalty(0.99).to(device)


In [ ]:
# Generator, Discriminator의 learning rate
learning_rate: float = 1e-3
# mapping_network의 learning rate
mapping_network_learning_rate: float = 1e-5
# Number of steps to accumulate gradient
# 이 숫자는 실질적으로 batch_size를 늘려주는 효과와 비슷함
gradient_accumulate_steps: int = 1
# Adam optimizer의 beta 파라미터 설정
adam_betas: Tuple[float, float] = (0.0, 0.99)
# 서로 다른 꼴 섞기
style_mixing_prob: float = 0.9

In [ ]:
# Discriminator, generator loss
discriminator_loss = DiscriminatorLoss().to(device)
generator_loss = GeneratorLoss().to(device)

# optimizer
discriminator_optimizer = torch.optim.Adam(discriminator.parameters(), lr=learning_rate, betas=adam_betas)
generator_optimizer = torch.optim.Adam(generator.parameters(), lr=learning_rate, betas=adam_betas)
mapping_network_optimizer = torch.optim.Adam(mapping_network.parameters(), lr=mapping_network_learning_rate, betas=adam_betas)


In [ ]:
# gradient penalty 계산하는 간격
lazy_gradient_penalty_interval: int = 4
# path length penalty 계산 간격
lazy_path_length_penalty_interval: int = 32
# 훈련 처음하는 시기엔 path length penalty 계산을 넘기기
lazy_path_penalty_after: int 5_000

# log로 생성된 그림들 보여주기
log_generated_interval: int = 500
# model을 얼마나 자주 저장할 것인가?
save_checkpoint_interval: int = 2_000

In [ ]:
if not os.path.exists("checkpoints"):
  os.makedirs("checkpoints")

In [ ]:
def get_w(batch_size: int):
  """
  표본
  이 표본은 서로 다른 지점을 넘나들고, $w_1$이 서로 다른 지점을 넘기 전에 generator blocks에 적용되고, $w_2$가 generator blocks에 적용된 후에 넘어감
  """
  # 꼴 섞기
  if torch.rand(()).item()< style_mixing_prob:
    # 무작위로 지점 넘나들기
    cross_over_point = int(torch.rand(()).item() * n_gen_blocks)
    # 표본 뽑기
    z2 = torch.randn(batch_size, d_latent).to(device)
    z1 = torch.randn(batch_size, d_latent).to(device)
    # w1, w2 얻기
    w1 = mapping_network(z1)
    w2 = mapping_network(z2)
    # w1, w2를 generator block에 적용하기 위해 ..
    w1 = w1[None, :, :].expand(cross_over_point, -1, -1)
    w2 = w2[None, :, :].expand(n_gen_blocks - cross_over_point, -1, -1)
    return torch.cat((w1, w2), dim=0)
  else:
    z = torch.randn(batch_size, d_latent).to(device)
    #
    w = mapping_network(z)

    return w[None, :, :].expand(n_gen_blocks, -1, -1)

def get_noise(batch_size: int):
  """

  """
  # 잡티 저장할 곳
  noise = []
  #
  resolution = 4
  # generator noise
  for i in range(n_gen_blocks):
    # 첫번째 block은 convolution $3 \times 3$개
    if i == 0:
      n1= None
    # Generator 잡티는 첫 번째 layer 끝나고 더하기
    else:
      n1 = torch.randn(batch_size, 1, resolution, resolution, device=device)
    # Generator 잡티를 두 번째 layer 끝나고 더하기
    n2= torch.randn(batch_size, 1, resolution, resolution, device=device)
    # noise tensor를 저장하기
    noise.append((n1, n2))
    # 다음 block의 해상도!!
    resolution *= 2

  return noise

def generate_images(batch_size: int):
  """
  그림 생성!!
  """
  w = get_w(batch_size)
  noise = get_noise(batch_size)

  images = geneator(w, noise)
  # 그림과 w 돌려주기
  return images, w



In [ ]:
def step(idx: int):
  """
  훈련 과정
  """
  # reset gradient
  discriminator_optimizer.zero_grad()

  # 'gradient_accumulate_steps'로 gradient 계산 가속
  for i in range(gradient_accumulate_steps):
    # generator로 얻은 그림 표본
    generated_images, _ = generate_images(batch_size)
    # Discriminator로 그림 구분
    fake_output = discriminator(generated_images.detach())

    # 그림 폴더에서 가져오기
    real_images = next(loader).to(device)
    # gradient 계산하기
    if (idx +1) % lazy_gradient_penalty_interval == 0:
      real_images.requires_grad_()

    # 실제 그림 discriminator로 그림 구분
    real_output = discriminator(real_images)

    # discriminator loss 계산
    real_loss, fake_loss = discriminator_loss(real_output, fake_output)
    disc_loss = real_loss + fake_loss
    # gradient penalty 계산
    if (idx +1) % lazy_gradient_penalty_interval == 0:
      # gradient penalty 계산
      gp = gradient_penalty(real_images, real_output)
      # coeff 적용하고, gradient penalty 더하기
      disc_loss = disc_loss + 0.5 * gradient_penalty_coefficient * gp * lazy_gradient_penalty_interval

    # gradient 계산
    disc_loss.backward()

  # gradient 일부 자르기(안정성)
  torch.nn.utils.clip_grad_norm_(discriminator.parameters(), max_norm=1.0)
  # optimizer step
  discriminator_optimizer.step()

  # reset gradient
  generator_optimizer.zero_grad()
  mapping_network_optimizer.zero_grad()

  # 'gradient_accumulate_steps'로 gradient 계산 가속
  for i in range(gradient_accumulate_steps):
    # generator로 얻은 그림 표본
    generated_images, w = generate_images(batch_size)
    # Discriminator로 그림 구분
    fake_output = discriminator(generated_images)

    # generator loss 얻기
    gen_loss = generator_loss(fake_output)
    # path length penalty 더하기
    if idx > lazy_path_penalty_after and (idx + 1) % lazy_path_penalty_interval == 0:
      # path length penalty 계산
      plp = path_length_penalty(w, generated_images)
      # nan 예외
      if not torch.isnan(plp):
        gen_loss = gen_loss + plp

    gen_loss.back_ward()

    # gradient 계산
    gen_loss.backward()
  # 안정성을 위해 gradient 일부 자르기
  torch.nn.utils.clip_grad_norm_(generator.parameters(), max_norm=1.0)
  torch.nn.utils.clip_grad_norm_(mapping_network.parameters(), max_norm=1.0)
  # optimizer
  generator_optimizer.step()
  mapping_network_optimizer.step()

  utils.save_image(
      torch.cat([generated_images[:6], real_images[:3]], dim=0),
      os.path.join('checkpoints', 'sample.png'),
      nrow=3,
      normalize=True,
      value_range=(-1, 1)
  )

  # 모델 저장
  if (idx + 1) % save_checkpoint_interval == 0:
    torch.save(generator.state_dict(), os.path.join('checkpoints', 'generator.pth'))
    torch.save(discriminator.state_dict(), os.path.join('checkpoints', 'discriminator.pth'))
    torch.save(mapping_network.state_dict(), os.path.join('checkpoints', 'mapping_network.pth'))


In [ ]:
# 훈련 시킬 숫자
training_steps: int = 150_000
# 훈련 루프
for i in tqdm(range(training_steps())):
  step(i)

